In [ ]:
import torch
import sys
import os
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import scipy
from datetime import datetime
import json

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
	sys.path.insert(0, module_path)
print(sys.path)

import Double_Pendulum.Learning.autoencoders as autoencoders
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.transforms as transforms
import Double_Pendulum.dynamics as dynamics
import Plotting.pendulum_plot as pendulum_plot

from functools import partial



%load_ext autoreload
%autoreload 2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
rp = robot_parameters.LUMPED_PARAMETERS
plotter = pendulum_plot.Anim_plotter(rp)


model_cw = False
model = autoencoders.Analytic_transformer(rp)

In [ ]:
K = torch.tensor([[10., 5.]]).to(device)

In [ ]:
def calculate_RxRy(q, q_d):
	Rx = rp["xa"] - rp["l0"] * torch.cos(q[0]) - rp["l1"] * torch.cos(q[1])
	Ry = rp["ya"] - rp["l0"] * torch.sin(q[0]) - rp["l1"] * torch.sin(q[1])
	return Rx, Ry

def calculate_RxRyDot(q, q_d):
	RxDot = rp["l0"] * torch.sin(q[0]) * q_d[0] + rp["l1"] * torch.sin(q[1]) * q_d[1]
	RyDot = -rp["l0"] * torch.cos(q[0]) * q_d[0] - rp["l1"] * torch.cos(q[1]) * q_d[1]
	return RxDot, RyDot
	
def calculate_RxRyDDot(q, q_d, q_dd_active, q_dd_passive):
	RxDDot_passive = (rp["l0"] * torch.cos(q[0]) * q_d[0]**2 + rp["l0"] * torch.sin(q[0]) * q_dd_passive[0] + 
					  rp["l1"] * torch.cos(q[1]) * q_d[1]**2 + rp["l1"] * torch.sin(q[1]) * q_dd_passive[1])
	RxDDot_active = rp["l0"] * torch.sin(q[0]) * q_dd_active[0] + rp["l1"] * torch.sin(q[1]) * q_dd_active[1]

	RyDDot_passive = (rp["l0"] * torch.sin(q[0]) * q_d[0]**2 - rp["l0"] * torch.cos(q[0]) * q_dd_passive[0] + 
					  rp["l1"] * torch.sin(q[1]) * q_d[1]**2 - rp["l1"] * torch.cos(q[1]) * q_dd_passive[1])
	RyDDot_active = - rp["l0"] * torch.cos(q[0]) * q_dd_active[0] - rp["l1"] * torch.cos(q[1]) * q_dd_active[1]

	return RxDDot_passive, RxDDot_active, RyDDot_passive, RyDDot_active

def calculate_N(Rx, Ry, RxDot, RyDot):
	N = Rx * RxDot + Ry * RyDot
	return N

def calculate_y(Rx, Ry):
	y = torch.sqrt(Rx**2 + Ry**2)
	return y

def calculate_yDot(y, N):
	yDot = 1/y * N
	return yDot

def calculate_Y(Rx, Ry, N):
	y = calculate_y(Rx, Ry)
	y_d = calculate_yDot(y, N)
	Y = torch.tensor([[y], [y_d]]).to(device)
	return Y

def calculate_alpha(y, N, Rx, Ry, RxDot, RyDot, RxDDot_passive, RyDDot_passive):
	alpha = -N**2/(y**3) + (1/y)*(RxDot**2 + RyDot**2 + Rx * RxDDot_passive + Ry * RyDDot_passive)
	return alpha

def calculate_beta(y, Rx, Ry, RxDDot_active, RyDDot_active):
	beta = (1/y) * (Rx * RxDDot_active + Ry * RyDDot_active)
	return beta

def calculate_v(Y_des, Y, K):
	v = K @ (Y_des - Y)
	return v

def calculate_u(v, alpha, beta):
	u = (1/beta) * (-alpha + v)
	return u





In [ ]:
xy_start = torch.tensor([0., -4.]).requires_grad_().to(device)
q_start = transforms.inverse_kinematics(xy_start, rp, is_clockwise=model_cw).unsqueeze(0)
q_d_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)
q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)

is_clockwise_start = transforms.check_clockwise(q_start.squeeze(0))

xy_des_real = torch.tensor([2., -2.]).requires_grad_().to(device)
q_des = transforms.inverse_kinematics(xy_des_real, rp, is_clockwise=model_cw).unsqueeze(0)
q_d_des = torch.tensor([[0., 0.]]).requires_grad_().to(device)
Rx_des, Ry_des = calculate_RxRy(q_des.squeeze(0), q_d_des.squeeze(0))
RxDot_des, RyDot_des = calculate_RxRyDot(q_des.squeeze(0), q_d_des.squeeze(0))
N_des = calculate_N(Rx_des, Ry_des, RxDot_des, RyDot_des)
Y_des = calculate_Y(Rx_des, Ry_des, N_des)


th_start = model.encoder_vmap(q_start)
th_d_start = (model.jacobian_enc(q_start) @ q_d_start.T).T

is_clockwise_des = transforms.check_clockwise(q_des.squeeze(0))

th_des = model.encoder_vmap(q_des)
th_d_des = (model.jacobian_enc(q_des) @ q_d_des.T).T

q_des_est = model.decoder_vmap(th_des, is_clockwise_des)
q_d_des_est = (model.jacobian_dec(th_des, clockwise=is_clockwise_des) @ th_d_des.T).T
xy_des_est, _ = transforms.forward_kinematics(rp, q_des_est[0])

#### Here comes the BOOM

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

th_series, th_d_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_naive_series, q_d_naive_series, q_dd_naive_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
u_series = torch.empty((0,1)).to(device)
tau_q_series = torch.empty((0,2)).to(device)

dt = 0.01
t_end = 15
t_series = torch.arange(0, t_end, dt)

q_real, q_d_real, q_dd_real = q_start, q_d_start, q_dd_start

is_clockwise = transforms.check_clockwise(q_start.squeeze(0))

for t in torch.arange(0, t_end, dt):
	t_string = "Time: [" + str(t.item().__round__(3)) + "/" + str(t_end) + ".0]"
	print(t_string)


	is_clockwise = transforms.check_clockwise(q_real.squeeze(0))

	q = q_real
	q_d = q_d_real

	th = model.encoder_vmap(q)
	th_d = (model.jacobian_enc(q) @ q_d.T).T
	
	q_hat = model.decoder_vmap(th, clockwise=is_clockwise)
	q_d_hat = (model.jacobian_dec(th) @ th_d.T).T

	""" Obtain dynamical matrices"""

	M_q, C_q, G_q = dynamics.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
	A_q = dynamics.input_matrix(rp, q.squeeze(0))


	""" Calculate normal form representation """

	M_q_inv = torch.pinverse(M_q)
	q_dd_active = (M_q_inv @ A_q).T
	q_dd_passive = (-M_q_inv @ (C_q @ q_d.T + G_q)).T


	Rx, Ry = calculate_RxRy(q.squeeze(0), q_d.squeeze(0))
	RxDot, RyDot = calculate_RxRyDot(q.squeeze(0), q_d.squeeze(0))
	RxDDot_passive, RxDDot_active, RyDDot_passive, RyDDot_active = calculate_RxRyDDot(q.squeeze(0), q_d.squeeze(0), q_dd_active.squeeze(0), q_dd_passive.squeeze(0))

	N = calculate_N(Rx, Ry, RxDot, RyDot)

	y = calculate_y(Rx, Ry)
	y_d = calculate_yDot(y, N)
	Y = torch.tensor([[y], [y_d]]).to(device)

	alpha = calculate_alpha(y, N, Rx, Ry, RxDot, RyDot, RxDDot_passive, RyDDot_passive)
	beta = calculate_beta(y, Rx, Ry, RxDDot_active, RyDDot_active)

	v = calculate_v(Y_des, Y, K)
	u = calculate_u(v, alpha, beta)

	""" Update the real system and apply latent control input. """

	M_q_real, C_q_real, G_q_real = dynamics.dynamical_matrices(rp, q_real.squeeze(0), q_d_real.squeeze(0))
	A_q_real = dynamics.input_matrix(rp, q_real.squeeze(0))

	tau_q_real = A_q_real * u
	q_dd_real = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_real).T)- G_q_real)).T  #  - C_damp @ ((q_d_real).T) 
	q_d_real = q_d_real + q_dd_real * dt
	q_real = q_real + q_d_real * dt
	q_real = transforms.wrap_to_pi(q_real)
	
	
	q_real_shifted = transforms.shift_q(q_real, clockwise=model_cw)
	th = model.encoder_vmap(q_real_shifted)
	#print(q_real)
	q_est = model.decoder_vmap(th, clockwise=transforms.check_clockwise(q_real.squeeze(0)))
	q_est = transforms.wrap_to_pi(q_est)
	th_d = (model.jacobian_enc(q_real) @ q_d_real.T).T
	q_d_est = (model.jacobian_dec(th, clockwise=is_clockwise) @ th_d.T).T


	""" Store data for plotting """

	th_series = torch.cat((th_series, th.detach()), dim=0)
	th_d_series = torch.cat((th_d_series, th_d.detach()), dim=0)
	
	q_naive_series = torch.cat((q_naive_series, q_real.detach()), dim=0)
	q_d_naive_series = torch.cat((q_d_naive_series, q_d_real.detach()), dim=0)
	q_dd_naive_series = torch.cat((q_dd_naive_series, q_dd_real.detach()), dim=0)


	u_series = torch.cat((u_series, u.detach()), dim=0)
	tau_q_series = torch.cat((tau_q_series, tau_q_real.T.detach()), dim=0)

	#print(is_clockwise)
	print("")

	
	


In [ ]:
def naive_to_xy(q_naive_series):
	xy_naive_series = torch.empty((0,2)).to(device)

	#for q_real, q_est, q_naive in zip(q_real_series, q_est_series, q_naive_series):
	for q_naive in q_naive_series:
		xy_naive, _ = transforms.forward_kinematics(rp, q_naive.squeeze(0))
		xy_naive_series = torch.cat((xy_naive_series, xy_naive.unsqueeze(0)))

	return (xy_naive_series)

In [ ]:
joint_power_series = tau_q_series * q_d_naive_series
power_series = torch.sum(joint_power_series, dim = 1)
total_power = torch.sum(power_series)
print(joint_power_series.size())
print(power_series.size())
print(total_power.item())

In [ ]:
def plot_naive(Kp_naive, Kd_naive, q_naive_series, th_naive_series, xy_naive_series, u_naive_series, dt, t_end):

	datasets_q = [
		{
			"name": "naive",
			"values": q_naive_series.cpu().detach().numpy(),
			"color": "tab:green"
		}
	]
	datasets_th = [
		{
			"name": "naive",
			"values": th_naive_series.cpu().detach().numpy(),
			"color": "tab:orange"
		}
	]

	datasets_xy = [
		{
			"name": "naive",
			"values": xy_naive_series.cpu().detach().numpy(),
			"color": "tab:green"
		}
	]

	datasets_u = [
		{
			"name": "naive",
			"values": u_naive_series.cpu().detach().numpy(),
			"color": "tab:orange"
		}
	]

	# Common labels for the plots.
	name_q = "q trajectory"
	name_th = "th trajectory"
	name_xy = "xy trajectory"
	t_series = torch.arange(0, t_end, dt)

	# Create an instance of ErrorPlotter.
	ep = pendulum_plot.Error_plotter(rp)

	# Prepare plot datasets for each column.
	# Each call groups a set of datasets to be drawn in one subplot column.
	column1 = ep.create_plot_dataset(t=t_series, datasets=datasets_q, reference=q_des, name=name_q)
	column2 = ep.create_plot_dataset(t=t_series, datasets=datasets_th, reference=th_des, name=name_th)
	column3 = ep.create_plot_dataset(t=t_series, datasets=datasets_xy, reference=xy_des_real.unsqueeze(0), name=name_xy)
	plot_datasets = [column1, column2, column3]

	print(column1["x"].size())

	file_name = "Error plot naive " + str(round(Kp_naive.item())) + "_" + str(round(Kd_naive.item())) + ".png"
	print(file_name)
	file_counter = 0

	output_path = os.path.join(save_dir, file_name)

	# Pass the list of columns (plot_dataset objects) to plot_multi.
	ep.plot_multi(plot_datasets=plot_datasets, save_path=output_path, axes_names = ["q", "th", "xy"])

In [ ]:
Kp_naive = K[0, 0]
Kd_naive = K[0, 1]
print(Kd_naive)
xy_naive_series = naive_to_xy(q_naive_series)
save_dir = "collocated_results/naive"
os.makedirs(save_dir, exist_ok=True)
plot_naive(Kp_naive, Kd_naive, q_naive_series, th_series, xy_naive_series, u_series, dt, t_end)

In [ ]:
stride = 3
pos_end_q_naive, pos_elbow_q_naive = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_naive_series[::stride])

frames_q_naive = plotter.frame_pendulum(pos_end_q_naive, pos_elbow_q_naive)

"""
data_real = {
	"frames": frames_q_real,
	"times": dt,
	"name": "q_real", 
	"arm_color": "tab:blue",
	"act_color": "tab:cyan"
}

data_est = {
	"frames": frames_q_est,
	"times": dt,
	"name": "q_est",
	"arm_color": "tab:orange", 
	"act_color": "tab:red"
} 
"""


data_naive = {
	"frames": frames_q_naive,
	"times": dt,
	"name": "q_naive",
	"arm_color": "tab:green", 
	"act_color": "tab:olive"
}

"""
ref_pos_real = {
	"pos": xy_des_real,
	"name": "real",
	"color": "tab:blue"
}

ref_pos_est = {
	"pos": xy_des_est,
	"name": "est",
	"color": "tab:orange"
}
"""

ref_pos_naive = {
	"pos": xy_des_real,
	"name": "naive",
	"color": "tab:green"
}


ref_poss = [ref_pos_naive]

frames_data = [data_naive]

name_rp = "RP:(" + str(rp["xa"]) + "," + str(rp["ya"]) + ")_"
name_ref = "ref:(" + str(xy_des_real[0].item()) + "," + str(xy_des_real[1].item()) + ")_"

model_type = "AL"

file_name = "t_end:[" + str(t_end) + "]_dt:[" + str(dt) + "]_stride:[" + str(stride) + "]_0.mp4"
file_counter = 0

output_path = os.path.join(save_dir, file_name)

while os.path.isfile(output_path):
	print("file name already exists")
	file_counter += 1
	file_name = file_name[:-6] + "_" + str(file_counter) + ".mp4"
	output_path = os.path.join(save_dir, file_name)

plotter.animate_pendulum(frames_data, ref_poss=ref_poss, plot_actuator=True, save_path=output_path, fps = 1/(dt*stride), dt = dt*stride)
#plotter.animate_pendulum(frames_data, ref_pos=None, plot_actuator=False, file_name=file_name, fps = 1/(dt*stride), dt = dt*stride)
